In [41]:
import numpy as np
import pandas as pd
from pathlib import Path
import re

In [42]:
RAW_DATA_PATH = Path("../data/raw/")
PROCESSED_PATH = Path("../data/processed/")

In [43]:
def get_all_user_ids(data_path):
    files = list(data_path.glob("HR_*.csv"))
    ids = [
        re.search(r"HR_(\d+).csv", f.name).group(1)
        for f in files
        if re.search(r"HR_(\d+).csv", f.name)
    ]
    return sorted(list(set(ids)))

In [44]:
def clean_hr_data(file_path):
    """
    STEP 1: CLEANING (Per User)
    - Fix timestamps, remove duplicates, filter impossible values.
    - NO feature engineering here.
    """
    # 1. Load & Sort
    df = pd.read_csv(file_path)
    df["HRTIME"] = pd.to_datetime(df["HRTIME"])
    df = df.sort_values("HRTIME")

    # 2. Clean (Drop NaNs, Filter Outliers 40-180bpm)
    df = df.dropna(subset=["HR"])
    df = df[(df["HR"] > 40) & (df["HR"] < 180)]

    # 3. Handle Duplicates (merged the rows into one single row by calculating the average.)
    df = df.groupby("HRTIME", as_index=False)["HR"].mean()
    df = df.sort_values("HRTIME")
    return df

In [45]:
def engineer_hr_features(df):
    """
    STEP 2: ENGINEERING (Per User)
    - Rolling Windows & Expanding Baselines.
    - Requires continuous time series (cannot be done after split).
    """

    # Set index for time-aware rolling
    df = df.set_index("HRTIME").sort_index()

    # Rolling Features (Short-term State - 5 min window)
    # min_periods=30 ensures we have at least 30 seconds of data
    df["mean_hr_5min"] = df["HR"].rolling("5min", min_periods=30).mean()
    df["hr_volatility_5min"] = df["HR"].rolling("5min", min_periods=30).std()

    # --- Cumulative Features (Long-term Baseline) ---
    df["hr_mean_total"] = df["HR"].expanding(min_periods=30).mean()
    df["hr_std_total"] = df["HR"].expanding(min_periods=30).std()
    df["hr_std_total"] = df["hr_std_total"].fillna(1)

    # --- Z-Score (Standardized Severity) ---
    df["hr_zscore"] = (df["HR"] - df["hr_mean_total"]) / (df["hr_std_total"] + 0.1)

    # Jumpiness (Short Term Variability)
    time_gap = df.index.to_series().diff().dt.total_seconds()
    df["hr_diff"] = df["HR"].diff().abs()
    df.loc[time_gap > 5, "hr_diff"] = np.nan  # Only diff if gap < 5s

    df["hr_jumpiness_5min"] = np.sqrt((df["hr_diff"] ** 2).rolling("5min", min_periods=30).mean())

    # Stress Ratio (Coefficient of Variation)
    df["stress_cv"] = df["hr_volatility_5min"] / (df["mean_hr_5min"] + 0.1)

    # Final Clean: Remove Warm-up NaNs
    df = df.dropna(subset=["mean_hr_5min"])

    # Drop bad dates (Year 2000 bug)
    df = df[df.index.year > 2021].copy()

    if df.empty:
        return None

    return df.reset_index()

In [46]:
def clean_sleep_data(file_path):
    """
    CLEANING (Per User)
    - Responsibilities: Load, Clean NaNs/Duplicates, Filter logical durations.
    - Returns: Clean DataFrame with valid timestamps and raw duration.
    """
    df = pd.read_csv(file_path)
    df["START"] = pd.to_datetime(df["START"])
    df["END"] = pd.to_datetime(df["END"])

    # Remove missing values
    df = df.dropna(subset=["START", "END"])
    # Clean Duplicates
    df = df.drop_duplicates(subset=["START", "END"])

    # Calculate Duration (Needed for filtering)
    df["duration_hours"] = (df["END"] - df["START"]).dt.total_seconds() / 3600

    # 3. Filter Logical Duration (15 min to 36 hours)
    df = df[(df["duration_hours"] > 0.25) & (df["duration_hours"] < 36)]

    # 4. Sort (Critical for rolling calculations later)
    df = df.sort_values("END")

    return df

In [47]:
def engineer_sleep_features(df):
    """
    ENGINEERING (Per User)
    - Responsibilities: Calculate Sleep Debt and Cumulative Debt.
    """

    # Rolling Debt Logic
    df["sleep_debt"] = 8.0 - df["duration_hours"]

    # Cumulative Debt (Rolling sum over last 3 sessions)
    # Group by a dummy key because this function runs per-user
    df["_temp_group"] = 1
    df["cum_sleep_debt"] = (
        df.groupby("_temp_group")["sleep_debt"]
        .rolling(3, min_periods=1)
        .sum()
        .reset_index(0, drop=True)
    )
    return df[["START", "END", "duration_hours", "cum_sleep_debt"]]

In [48]:
def process_continuous_features(uid, raw_path):
    """
    Aggregates HR and Sleep data for a user regardless of PVT existence.
    Anchor: Heart Rate timestamps.
    """
    try:
        # 1. Define Paths
        hr_file = raw_path / f"HR_{uid}.csv"
        sleep_file = raw_path / f"sleep_{uid}.csv"

        if not hr_file.exists() or not sleep_file.exists():
            return None

        # --- 2. HR PROCESSING ---
        df_hr_clean = clean_hr_data(hr_file)
        df_hr_eng = engineer_hr_features(df_hr_clean)
        if df_hr_eng is None:
            return None

        # --- 3. SLEEP PROCESSING ---
        df_sleep_clean = clean_sleep_data(sleep_file)
        df_sleep_eng = engineer_sleep_features(df_sleep_clean)

        # Rename for merging
        df_sleep_merge = df_sleep_eng.rename(columns={"END": "last_sleep_end"})

        # --- 4. MERGE (HR as Anchor) ---
        # We merge sleep data onto HR data based on the HRTIME
        df_final = pd.merge_asof(
            df_hr_eng.sort_values("HRTIME"),
            df_sleep_merge.sort_values("last_sleep_end"),
            left_on="HRTIME",
            right_on="last_sleep_end",
            direction="backward",
        )

        # --- 5. CALCULATE TIME-BASED FEATURES ---
        # Hours since last sleep
        df_final["hours_awake"] = (
            df_final["HRTIME"] - df_final["last_sleep_end"]
        ).dt.total_seconds() / 3600

        # Circadian Phase (based on current HR time)
        hr_hour = df_final["HRTIME"].dt.hour + (df_final["HRTIME"].dt.minute / 60)
        df_final["circadian_sin"] = np.sin(2 * np.pi * hr_hour / 24)
        df_final["circadian_cos"] = np.cos(2 * np.pi * hr_hour / 24)

        # Sleep Inertia
        df_final["sleep_inertia_idx"] = 1 / (df_final["hours_awake"] + 0.1)

        # --- 6. CLEANUP ---
        # Add user_id and drop rows where we don't have sleep history
        df_final["user_id"] = uid
        df_final = df_final.dropna(subset=["hours_awake", "cum_sleep_debt"])

        # FIX 1: Ensure HRTIME is the index and sorted before resampling
        df_final = df_final.set_index("HRTIME").sort_index()
        df_final = df_final.resample("1min").mean(numeric_only=True).reset_index()

        df_final["user_id"] = uid

        return df_final.dropna()

    except Exception as e:
        print(f"Error processing {uid}: {e}")
        return None

In [49]:
all_continuous_data = []
user_ids = get_all_user_ids(RAW_DATA_PATH)
print(f"Found {len(user_ids)} users.")
for uid in user_ids:
    user_continuous = process_continuous_features(uid, RAW_DATA_PATH)
    if user_continuous is not None:
        all_continuous_data.append(user_continuous)

df_continuous_total = pd.concat(all_continuous_data, ignore_index=True)
df_continuous_total.to_parquet(PROCESSED_PATH / "continuous_features.parquet")

Found 29 users.


In [51]:
print(df_continuous_total.head())
print(len(df_continuous_total))

               HRTIME          HR  mean_hr_5min  hr_volatility_5min  \
0 2023-04-14 19:44:00   88.631579     87.776873            5.593524   
1 2023-04-14 19:45:00   91.315789     87.911509            4.714402   
2 2023-04-14 19:46:00  118.098039     95.160720           12.303997   
3 2023-04-14 19:47:00  137.421053    104.716552           19.269933   
4 2023-04-14 19:48:00  111.687500    110.122351           19.809114   

   hr_mean_total  hr_std_total  hr_zscore   hr_diff  hr_jumpiness_5min  \
0     103.128852     12.086594  -1.189605  1.222222           1.973392   
1     103.119211     12.088983  -0.968380  1.388889           1.714790   
2     103.123081     12.090308   1.228369  1.470588           1.674470   
3     103.148566     12.117112   2.805460  1.324324           1.620771   
4     103.168622     12.136320   0.696196  1.500000           1.573572   

   stress_cv  duration_hours  cum_sleep_debt  hours_awake  circadian_sin  \
0   0.063657        2.766667        5.233333     2.7

In [54]:
results = pd.read_parquet(
    "/Users/amyshih/Desktop/python/fatigue_prediction/data/predictions/fatigue_predictions.parquet"
)
print(results.head())

                HRTIME user_id  fatigue_probability  fatigue_event_alert  \
0  2023-04-14 19:44:00    3369             0.388276                    1   
1  2023-04-14 19:45:00    3369             0.388276                    1   
2  2023-04-14 19:46:00    3369             0.378783                    1   
3  2023-04-14 19:47:00    3369             0.420439                    1   
4  2023-04-14 19:48:00    3369             0.381330                    1   

     status  
0  Fatigued  
1  Fatigued  
2  Fatigued  
3  Fatigued  
4  Fatigued  
